# Setup

**Imports**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from loadmydata.load_human_locomotion import (
    load_human_locomotion_dataset,
    get_code_list,
)

**Utility functions**

In [74]:
def fig_ax(figsize=(15, 5)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.autoscale(enable=True, axis='x', tight=True)
    return fig, ax

# Spectral feature

## Question 6

In [70]:
def gamma_hat(signal):
    signal = signal - signal.mean()
    N = len(signal)
    corr = np.correlate(signal, signal, mode="full")[N-1:] / N
    return corr

def J(signal):
    N = len(signal)
    signal = signal - signal.mean()
    S = np.fft.fft(signal)
    
    return (np.abs(1/np.sqrt(N) * S)**2)[:N//2 + 1]


In [71]:
### Sample autocovariances ###

Ns = [200,500,1000]
N_sim = 100

for N in Ns:
    signals = np.random.rand(N_sim, N)
    gamma = np.zeros((N_sim,N))
    Jf = np.zeros((N_sim, N//2+1))
    for i,signal in enumerate(signals):
        gamma[i] = gamma_hat(signal)
        Jf[i] = J(signal)


    mean = gamma.mean(axis=0)
    std = gamma.std(axis=0)
    taus = np.arange(N)

    mean_freq_powe = Jf.mean(axis=0)
    std_freq_powe = Jf.std(axis=0)
    freqs = np.arange(N//2+1)

    plt.figure(figsize=(12, 6))
    plt.plot(taus, mean, color='C0')
    plt.fill_between(taus, mean - std, mean + std, color='C0', alpha=0.3)
    plt.xlabel("τ (lag)")
    plt.ylabel(r"$\hat{\gamma}(\tau)$")
    plt.title(f"Estimated autocovariance (N={N})")
    plt.tight_layout()
    plt.savefig(f"figures/gamma_N{N}.png", dpi=300)
    plt.close()  

    plt.figure(figsize=(12, 6))
    plt.plot(freqs, mean_freq_powe, color='C0')
    plt.fill_between(freqs, mean_freq_powe - std_freq_powe, mean_freq_powe + std_freq_powe, color='C0', alpha=0.3)
    plt.xlabel("f (frequency)")
    plt.ylabel(r"$|J(f_k)|^2$")
    plt.title(f"Estimated periodogram (N={N})")
    plt.tight_layout()
    plt.savefig(f"figures/Jfk**2_N{N}.png", dpi=300)
    plt.close()  




## Question 9

# Dynamic time warping (DTW)

## Data

This data set consists of signals collected with inertial measurement units (accelerometer+gyroscope), from 230 subjects undergoing a fixed protocol:
- standing still,
- walking 10 m,
- turning around,
- walking back,
- stopping.

In this assignment, we only consider the vertical acceleration of the left foot and all signals are truncated to 20 seconds (as a result, they all have same length). Signals are sampled at 100 Hz.

The measured population is composed of healthy subjects as well as patients with neurological or orthopedic disorders.

The start and end time stamps of thousands of footsteps are available.

The data are part of a larger data set described in [1].

[1] Truong, C., Barrois-Müller, R., Moreau, T., Provost, C., Vienne-Jumeau, A., Moreau, A., Vidal, P.-P., Vayatis, N., Buffat, S., Yelnik, A., Ricard, D., & Oudre, L. (2019). A data set for the study of human locomotion with inertial measurements units. Image Processing On Line (IPOL), 9.

**The task** is to classify footsteps in healthy/non-healthy.

The following cell defines the training set `(X_train, y_train)` and testing set `(X_test, y_test)`.

In [3]:
subset_indexes_train = [95, 619, 441, 149, 951, 803, 214, 34, 37, 630]
subset_indexes_test = [683, 259, 59, 387, 634]

code_list = get_code_list()

X_train = list()  # list of footstep signals
y_train = list()  # list of pathologies (the "labels")

for code in np.take(code_list, subset_indexes_train):
    single_trial = load_human_locomotion_dataset(code)
    signal = single_trial.signal.LAZ.to_numpy()  # keeping only one dimension (from the left sensor)
    steps = single_trial.left_steps
    pathology = single_trial.metadata["PathologyGroup"]
    label = 0 if pathology=="Healthy" else 1  # 0: healthy, 1: non-healthy
    for (start, end) in steps:
        X_train.append(signal[start:end])
        y_train.append(label)

        
X_test = list()  # list of footstep signals
y_test = list()  # list of pathologies (the "labels")

for code in np.take(code_list, subset_indexes_test):
    single_trial = load_human_locomotion_dataset(code)
    signal = single_trial.signal.LAZ.to_numpy()  # keeping only one dimension (from the left sensor)
    steps = single_trial.left_steps
    pathology = single_trial.metadata["PathologyGroup"]
    label = 0 if pathology=="Healthy" else 1  # 0: healthy, 1: non-healthy
    for (start, end) in steps:
        X_test.append(signal[start:end])
        y_test.append(label)

AssertionError: The code 17-1 cannot be found in the data set.

## Question 10

## Question 11